In [ ]:
import netCDF4 as nc
import torch
import time
import os
import pandas as pd
import numpy as np
from numpy import ma
from scipy.interpolate import griddata
from scipy.interpolate import CloughTocher2DInterpolator

In [ ]:
SODA_train_path = 'enso_round1_train_20210201/SODA_train.nc'
SODA_train = nc.Dataset(SODA_train_path)
SODA_label_path = 'enso_round1_train_20210201/SODA_label.nc'
SODA_label = nc.Dataset(SODA_label_path)

CMIP_train_path = 'enso_round1_train_20210201/CMIP_train.nc'
CMIP_train = nc.Dataset(CMIP_train_path)
CMIP_label_path = 'enso_round1_train_20210201/CMIP_label.nc'
CMIP_label = nc.Dataset(CMIP_label_path)

fea_names = ["sst", "t300", "ua", "va"]
label_name = "nino"

In [ ]:
all_vars = SODA_train.variables.keys()   #获取所有变量名称
print(SODA_train.variables)
all_vars = SODA_label.variables.keys()   #获取所有变量名称
print(SODA_label.variables)

In [ ]:
print(CMIP_train.variables['sst'][0,7,0,:])
print(CMIP_train.variables['sst'][0,7,1,:])
print(CMIP_train.variables['sst'][0,7,2,:])
print(CMIP_train.variables['sst'][0,7,3,:])
print(CMIP_train.variables['sst'][0,7,4,:])
print(CMIP_train.variables['sst'][0,7,5,:])
print(CMIP_train.variables['sst'][0,7,6,:])
print(CMIP_train.variables['sst'][0,7,7,:])
print(CMIP_train.variables['sst'][0,7,8,:])
print(CMIP_train.variables['sst'][0,7,9,:])
print(CMIP_train.variables['sst'][0,7,10,:])
print(CMIP_train.variables['sst'][0,7,11,:])
print(CMIP_train.variables['sst'][0,7,12,:])
print(CMIP_train.variables['sst'][0,7,13,:])
print(CMIP_train.variables['sst'][0,7,14,:])
print(CMIP_train.variables['sst'][0,7,15,:])
print(CMIP_train.variables['sst'][0,7,16,:])
print(CMIP_train.variables['sst'][0,7,17,:])
print(CMIP_train.variables['sst'][0,7,18,:])
print(CMIP_train.variables['sst'][0,7,19,:])
print(CMIP_train.variables['sst'][0,7,20,:])
print(CMIP_train.variables['sst'][0,7,21,:])
print(CMIP_train.variables['sst'][0,7,22,:])
print(CMIP_train.variables['sst'][0,7,23,:])

In [ ]:
#print(SODA_train.variables['sst'][0,0,:,:])
#print(SODA_train.variables['va'][99,35,:,:])
print(SODA_label.variables['nino'][:])
soda_label_np = np.zeros(102*12)
soda_train_np = np.zeros((4,102*12,24,72))
for i, fea in enumerate(fea_names):
    soda_train_temp = np.zeros((102*12,24,72))
    for j in range(100):
        if(j==99):
            soda_train_temp[12*j:12*(j+3)] = SODA_train.variables[fea][j, 0:36, :, :]
        else:
            soda_train_temp[12*j:12*(j+1)] = SODA_train.variables[fea][j, 0:12, :, :]
    soda_train_np[i] = soda_train_temp
#print(soda_train_np)
for j in range(100):
    if(j==99):
        soda_label_np[12*j:12*(j+3)] = SODA_label.variables['nino'][j, 0:36]
    else:
        soda_label_np[12*j:12*(j+1)] = SODA_label.variables['nino'][j, 0:12]
print(soda_label_np)

In [ ]:
#print(CMIP_train.variables['sst'][0,0,:,:])
#print(CMIP_train.variables['va'][99,35,:,:])
#print(CMIP_label.variables['nino'][:])
cmip6_label_np = np.zeros((15,153*12))
cmip6_train_np = np.zeros((15,4,153*12,24,72))
for k in range(15):
    #print(CMIP_train.variables['sst'][151*k:151*(k+1),:,:,:])
    cmip_train_np = np.zeros((4,153*12,24,72))
    for i, fea in enumerate(fea_names):
        cmip_train_temp = np.zeros((153*12,24,72))
        for j in range(151):
            if(j==150):
                cmip_train_temp[12*j:12*(j+3)] = CMIP_train.variables[fea][k*151+j, 0:36, :, :]
            else:
                cmip_train_temp[12*j:12*(j+1)] = CMIP_train.variables[fea][k*151+j, 0:12, :, :]
        cmip_train_np[i] = cmip_train_temp
    cmip6_train_np[k] = cmip_train_np
    #print(cmip_train_np)
    
    #print(CMIP_label.variables['nino'][151*k:151*(k+1),:])
    cmip_label_np = np.zeros(153*12)
    for j in range(151):
        if(j==150):
            cmip_label_np[12*j:12*(j+3)] = CMIP_label.variables['nino'][k*151+j, 0:36]
        else:
            cmip_label_np[12*j:12*(j+1)] = CMIP_label.variables['nino'][k*151+j, 0:12]
    #print(cmip_label_np)
    cmip6_label_np[k] = cmip_label_np


In [ ]:
#print(CMIP_train.variables['sst'][0,0,:,:])
#print(CMIP_train.variables['va'][99,35,:,:])
#print(CMIP_label.variables['nino'][:])
cmip5_label_np = np.zeros((17,142*12))
cmip5_train_np = np.zeros((17,4,142*12,24,72))
for k in range(17):
    #print(CMIP_train.variables['sst'][2265+140*k:2265+140*(k+1),:,:,:])
    cmip_train_np = np.zeros((4,142*12,24,72))
    for i, fea in enumerate(fea_names):
        cmip_train_temp = np.zeros((142*12,24,72))
        for j in range(140):
            if(j==139):
                cmip_train_temp[12*j:12*(j+3)] = CMIP_train.variables[fea][2265+k*140+j, 0:36, :, :]
            else:
                cmip_train_temp[12*j:12*(j+1)] = CMIP_train.variables[fea][2265+k*140+j, 0:12, :, :]
        cmip_train_np[i] = cmip_train_temp
    cmip5_train_np[k] = cmip_train_np
    #print(cmip_train_np)
    
    print(CMIP_label.variables['nino'][2265+140*k:2265+140*(k+1),:])
    cmip_label_np = np.zeros(142*12)
    for j in range(140):
        if(j==139):
            cmip_label_np[12*j:12*(j+3)] = CMIP_label.variables['nino'][k*140+j, 0:36]
        else:
            cmip_label_np[12*j:12*(j+1)] = CMIP_label.variables['nino'][k*140+j, 0:12]
    print(cmip_label_np)
    cmip5_label_np[k] = cmip_label_np


In [ ]:
for i in range(cmip6_train_np.shape[0]):
    for j in range(2):
        for k in range(cmip6_train_np.shape[2]):
            for l in range(cmip6_train_np.shape[3]):
                for m in range(cmip6_train_np.shape[4]):
                    if np.isnan(cmip6_train_np[i,j,k,l,m]):
                        print('{5}:{0},{1},{2},{3},{4}'.format(i,j,k,l,m,cmip6_train_np[i,j,k,l,m]))

In [ ]:
np.set_printoptions(threshold=100000)
print(cmip6_train_np[12,1,1835,:,:])


In [ ]:
loc = []
val = []
for i in range(cmip6_train_np.shape[3]):
    for j in range(cmip6_train_np.shape[4]):
        if(np.isnan(cmip6_train_np[12,1,1835,i,j])==False):
            temp = []
            temp.append(i)
            temp.append(j)
            loc.append(temp)
            val.append(cmip6_train_np[12,1,1835,i,j])
loc = np.array(loc)
val = np.array(val)
#print(loc)
#print(val)
x, y = np.mgrid[0:23:24j, 0:71:72j]
grid = griddata(loc, val, (x,y), method="cubic",fill_value=0)
#grid = CloughTocher2DInterpolator(loc, val)
#grid = interp2d(x, y, cmip6_train_np[12,1,1835,:,:], kind='cubic')
print(grid)

In [ ]:
import torch.nn as nn
import torch


class ConvLSTMCell(nn.Module):

    def __init__(self, input_dim, hidden_dim, kernel_size, bias):
        """
        Initialize ConvLSTM cell.
        Parameters
        ----------
        input_dim: int
            Number of channels of input tensor.
        hidden_dim: int
            Number of channels of hidden state.
        kernel_size: (int, int)
            Size of the convolutional kernel.
        bias: bool
            Whether or not to add the bias.
        """

        super(ConvLSTMCell, self).__init__()

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim

        self.kernel_size = kernel_size
        self.padding = kernel_size[0] // 2, kernel_size[1] // 2
        self.bias = bias

        self.conv = nn.Conv2d(in_channels=self.input_dim + self.hidden_dim,
                              out_channels=4 * self.hidden_dim,
                              kernel_size=self.kernel_size,
                              padding=self.padding,
                              bias=self.bias)

    def forward(self, input_tensor, cur_state):
        h_cur, c_cur = cur_state

        combined = torch.cat([input_tensor, h_cur], dim=1)  # concatenate along channel axis

        combined_conv = self.conv(combined)
        cc_i, cc_f, cc_o, cc_g = torch.split(combined_conv, self.hidden_dim, dim=1)
        i = torch.sigmoid(cc_i)
        f = torch.sigmoid(cc_f)
        o = torch.sigmoid(cc_o)
        g = torch.tanh(cc_g)

        c_next = f * c_cur + i * g
        h_next = o * torch.tanh(c_next)

        return h_next, c_next

    def init_hidden(self, batch_size, image_size):
        height, width = image_size
        return (torch.zeros(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device),
                torch.zeros(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device))


class ConvLSTM(nn.Module):

    """
    Parameters:
        input_dim: Number of channels in input
        hidden_dim: Number of hidden channels
        kernel_size: Size of kernel in convolutions
        num_layers: Number of LSTM layers stacked on each other
        batch_first: Whether or not dimension 0 is the batch or not
        bias: Bias or no bias in Convolution
        return_all_layers: Return the list of computations for all layers
        Note: Will do same padding.
    Input:
        A tensor of size B, T, C, H, W or T, B, C, H, W
    Output:
        A tuple of two lists of length num_layers (or length 1 if return_all_layers is False).
            0 - layer_output_list is the list of lists of length T of each output
            1 - last_state_list is the list of last states
                    each element of the list is a tuple (h, c) for hidden state and memory
    Example:
        >> x = torch.rand((32, 10, 64, 128, 128))
        >> convlstm = ConvLSTM(64, 16, 3, 1, True, True, False)
        >> _, last_states = convlstm(x)
        >> h = last_states[0][0]  # 0 for layer index, 0 for h index
    """

    def __init__(self, input_dim, hidden_dim, kernel_size, num_layers,
                 batch_first=False, bias=True, return_all_layers=False):
        super(ConvLSTM, self).__init__()

        self._check_kernel_size_consistency(kernel_size)

        # Make sure that both `kernel_size` and `hidden_dim` are lists having len == num_layers
        kernel_size = self._extend_for_multilayer(kernel_size, num_layers)
        hidden_dim = self._extend_for_multilayer(hidden_dim, num_layers)
        if not len(kernel_size) == len(hidden_dim) == num_layers:
            raise ValueError('Inconsistent list length.')

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.kernel_size = kernel_size
        self.num_layers = num_layers
        self.batch_first = batch_first
        self.bias = bias
        self.return_all_layers = return_all_layers

        cell_list = []
        for i in range(0, self.num_layers):
            cur_input_dim = self.input_dim if i == 0 else self.hidden_dim[i - 1]

            cell_list.append(ConvLSTMCell(input_dim=cur_input_dim,
                                          hidden_dim=self.hidden_dim[i],
                                          kernel_size=self.kernel_size[i],
                                          bias=self.bias))

        self.cell_list = nn.ModuleList(cell_list)

    def forward(self, input_tensor, hidden_state=None):
        """
        Parameters
        ----------
        input_tensor: todo
            5-D Tensor either of shape (t, b, c, h, w) or (b, t, c, h, w)
        hidden_state: todo
            None. todo implement stateful
        Returns
        -------
        last_state_list, layer_output
        """
        if not self.batch_first:
            # (t, b, c, h, w) -> (b, t, c, h, w)
            input_tensor = input_tensor.permute(1, 0, 2, 3, 4)

        b, _, _, h, w = input_tensor.size()

        # Implement stateful ConvLSTM
        if hidden_state is not None:
            raise NotImplementedError()
        else:
            # Since the init is done in forward. Can send image size here
            hidden_state = self._init_hidden(batch_size=b,
                                             image_size=(h, w))

        layer_output_list = []
        last_state_list = []

        seq_len = input_tensor.size(1)
        cur_layer_input = input_tensor

        for layer_idx in range(self.num_layers):

            h, c = hidden_state[layer_idx]
            output_inner = []
            for t in range(seq_len):
                h, c = self.cell_list[layer_idx](input_tensor=cur_layer_input[:, t, :, :, :],
                                                 cur_state=[h, c])
                output_inner.append(h)

            layer_output = torch.stack(output_inner, dim=1)
            cur_layer_input = layer_output

            layer_output_list.append(layer_output)
            last_state_list.append([h, c])

        if not self.return_all_layers:
            layer_output_list = layer_output_list[-1:]
            last_state_list = last_state_list[-1:]

        return layer_output_list, last_state_list

    def _init_hidden(self, batch_size, image_size):
        init_states = []
        for i in range(self.num_layers):
            init_states.append(self.cell_list[i].init_hidden(batch_size, image_size))
        return init_states

    @staticmethod
    def _check_kernel_size_consistency(kernel_size):
        if not (isinstance(kernel_size, tuple) or
                (isinstance(kernel_size, list) and all([isinstance(elem, tuple) for elem in kernel_size]))):
            raise ValueError('`kernel_size` must be tuple or list of tuples')

    @staticmethod
    def _extend_for_multilayer(param, num_layers):
        if not isinstance(param, list):
            param = [param] * num_layers
        return param